In [ ]:
#hide
%load_ext autoreload
%autoreload 2

In [ ]:
#hide
from nnanno.core import *
from nnanno.sample import *
from nnanno.inference import *

# nnanno

> Sampling, annotating and making predictions on the Newspaper Navigator dataset  

What is Newspaper Navigator?

## Install

`pip install nnanno`

## Programming Historian Data Preparation 

## How to use

Fill me in please! Don't forget code examples:

In [ ]:
get_json_url(1850, 'ads')

'https://news-navigator.labs.loc.gov/prepackaged/1850_ads.json'

In [ ]:
sampler = nnSampler()
df = sampler.create_sample(2, end_year=1851)

In [ ]:
df['page_url'].head(1)

0    https://chroniclingamerica.loc.gov/data/batches/ncu_hawk_ver02/data/sn84026472/00416156360/1850052201/0289.jp2
Name: page_url, dtype: category
Categories (2, object): ['https://chroniclingamerica.loc.gov/data/batches/ncu_hawk_ver02/data/sn84026472/00416156360/1850052201/0289.jp2', 'https://chroniclingamerica.loc.gov/data/batches/ohi_ingstad_ver01/data/sn85026051/00296027029/1850082401/0053.jp2']

## Annotate samples

# Inference

In [ ]:
from fastai.vision.all import *
dls = ImageDataLoaders.from_csv('../ph/ads/', 'ads_upsampled.csv',folder='images', fn_col='file', label_col='label',item_tfms=Resize(64,ResizeMethod.Squish))
learn = cnn_learner(dls, resnet18, metrics=F1Score())
learn.fine_tune(1)

epoch,train_loss,valid_loss,f1_score,time
0,1.191004,0.606111,0.800000,00:09


epoch,train_loss,valid_loss,f1_score,time
0,0.535960,0.488017,0.800000,00:13


In [ ]:
predictor = nnPredict(learn,try_gpu=False)

In [ ]:
predictor.predict_sample('ads','testinference',0.01,end_year=1850)

In [ ]:
df = pd.read_json('testinference/1850.json')
df.iloc[:5,-3:]

,pred_decoded,illustrations_prob,text-only_prob
0,1,0.114718,0.885282
1,1,0.084845,0.915155
2,0,0.999954,0.000046
3,1,0.012195,0.987805
4,0,0.591514,0.408486
